In [18]:
import os
import math
nombreDataSet = "datasetP600"

# Indicar las direcciones de los ficheros

direccionNuestroValidacion   = f"{nombreDataSet}/labels/val"
direccionTrain              = f"{nombreDataSet}/labels/train"
direccionNuestroPredicciones = f"{nombreDataSet}/estadisticas/verdaderoPositivo"
direccionFalsoPositivo       = f"{nombreDataSet}/estadisticas/falsoPositivo"
direccionFalsoNegativo       = f"{nombreDataSet}/estadisticas/falsoNegativo"
direccionYoloPredicciones    = f"{nombreDataSet}/predict/"
direccionCalcularPartes      = f"{nombreDataSet}/estadisticas"
direccionYoloPrediccionestrain = f"{nombreDataSet}/predictTrain/"


ficherosValidacion = os.listdir( direccionNuestroValidacion   )

ficheroYoloPredicciones = os.listdir( direccionYoloPredicciones )
ficheroYoloPrediccionesTrain = os.listdir(direccionYoloPrediccionestrain)


In [19]:
def calcularArea(cuadrado:"list"):
    return float(cuadrado[3]) * float(cuadrado[4])

def errorArea( cuadradoVal:"list",cuadradoPrediccion:"list" ):
    return abs( calcularArea(cuadradoPrediccion) - calcularArea(cuadradoVal) )

def errorLocalizacion( cuadradoVal:"list",cuadradoPrediccion:"list" ):
    return math.sqrt( ( float(cuadradoVal[1]) - float(cuadradoPrediccion[1]) )**2 
                    + ( float(cuadradoVal[2]) - float(cuadradoPrediccion[2]) )**2 ) 

def cumpleEjeW(cuadradoVal:"list",cuadradoPrediccion:"list"):
    distancia = abs( float( cuadradoVal[1] ) - float(cuadradoPrediccion[1]) )
    distaniciaMaxima = (float( cuadradoVal[3] ) + float( cuadradoPrediccion[3] ))/2
    return distancia <= distaniciaMaxima

def cumpleEjeH(cuadradoVal:"list",cuadradoPrediccion:"list"):
    distancia = abs( float( cuadradoVal[2] ) - float(cuadradoPrediccion[2]) )
    distaniciaMaxima = (float( cuadradoVal[4] ) + float( cuadradoPrediccion[4] ))/2
    return distancia <= distaniciaMaxima

def estaDentro(cuadradoVal:"list",cuadradoPrediccion:"list"):
    return cumpleEjeH(cuadradoVal,cuadradoPrediccion) and cumpleEjeW(cuadradoVal,cuadradoPrediccion)


In [20]:
from typing import Counter
import os

def leer_archivo(archivo):
    """
    Lee el fichero con las anotaciones
    """
    cajas = []
    with open(archivo, "r", encoding="utf-8") as f:
        for linea in f:
            partes = linea.strip().split()
            clase = int(partes[0])
            x,y,w,h = map(float, partes[1:])
            cajas.append((clase, x, y, w, h))
    return cajas


# Saber la cantidad de falsos positivos y falsos negativos

def evaluar_imagenes_error(archivo_pred):
    """
    Divide las imagenes por cada parte
    """
    pred = leer_archivo(archivo_pred)
    pred_personas = [p for p in pred if p[0]==0]

    pred_partes = [p for p in pred if p[0] != 0]


    personas_dicc = {}

    for persona in pred_personas:

        _, x, y, w, h = persona
        xmin =  x - w/2
        ymin =  x - h/2
        xmax = x + w/2
        ymax = y + h/2

        acierto = []

        for partes in pred_partes:
            clase, x_centro, y_centro, _, _ = partes

            if (xmin < x_centro and ymin < y_centro and xmax > x_centro and ymax > y_centro):
                acierto.append(clase)

        personas_dicc[persona] = Counter(acierto)

    return personas_dicc

def evaluar_imagenes_error_sinArchivos(archivo_pred, pred_personas_entrada):
    """
    Divide las imagenes por cada parte sin el fichero original para las predicciones de personas
    """
    pred = leer_archivo(archivo_pred)
    pred_personas = pred_personas_entrada

    pred_partes = [p for p in pred if p[0] != 0]

    personas_dicc = {}

    for persona in pred_personas:

        _, x, y, w, h = persona
        xmin =  x - w/2
        ymin =  x - h/2
        xmax = x + w/2
        ymax = y + h/2

        acierto = []

        for partes in pred_partes:
            clase, x_centro, y_centro, _, _ = partes

            if (xmin < x_centro and ymin < y_centro and xmax > x_centro and ymax > y_centro):
                acierto.append(clase)

        personas_dicc[persona] = Counter(acierto)

    return personas_dicc

def ponderacion_personas(dicc_pers, umbral=   0.7 ,ponderacion =  {1:1.5, 2:0.6, 3: 0.6,4:1, 5:0.8, 6:1, 7:0.8, 8:2.0}  ):
    """
    Cálculo de las ponderaciones de cada persona en el diccionario
    """
    #ponderacion = {1:1.5, 2:0.6, 3: 0.6,4:1, 5:0.8, 6:1, 7:0.8, 8:2.0}
    
    verdaderos_positivos = set()
    falsos_positivos = set()
    falsos_negativos = set()

    for persona, partes in dicc_pers.items():
        if not partes:
            falsos_negativos.add(persona)
        else:
            puntos = sum(ponderacion[k] * partes.get(k,0) for k in ponderacion)
        
            if puntos >= umbral:
                verdaderos_positivos.add(persona)
            else:
                falsos_positivos.add(persona)

    return verdaderos_positivos, falsos_positivos, falsos_negativos

def guardar_personas(nombre_archivo, personas):
  """
  Guardar los datos en un fichero
  """
  with open(nombre_archivo, "w", encoding="utf-8") as f:
    for persona in personas:
      linea = " ".join(map(str, persona))
      f.write(linea + "\n")

def estadisticas(fichero ,umbral=   0.7 ,ponderacion =  {1:1.5, 2:0.6, 3: 0.6,4:1, 5:0.8, 6:1, 7:0.8, 8:2.0}):
    """
    Usa las funciones anteriores para sacar estadísticas
    """
    res = evaluar_imagenes_error(fichero)
    VP, FP, FN = ponderacion_personas(res ,umbral ,ponderacion  )
    return VP, FP, FN

def crear_estadisticas(ruta , rutaDepositar , umbral=   0.7 ,ponderacion =  {1:1.5, 2:0.6, 3: 0.6,4:1, 5:0.8, 6:1, 7:0.8, 8:2.0}):
    """
    Crea los ficheros con los datos calculados
    """
    carpeta_VP = f"{rutaDepositar}/verdaderoPositivo/"
    carpeta_FP = f"{rutaDepositar}/falsoPositivo/"
    carpeta_FN = f"{rutaDepositar}/falsoNegativo/"

    os.makedirs(carpeta_VP, exist_ok=True)
    os.makedirs(carpeta_FP, exist_ok=True)
    os.makedirs(carpeta_FN, exist_ok=True)

    for nombre_archivo in os.listdir(ruta):
        ruta_completa = os.path.join(ruta, nombre_archivo)
        if os.path.isfile(ruta_completa):
            VP, FP, FN = estadisticas(ruta_completa , umbral ,ponderacion )

            guardar_personas(os.path.join(carpeta_VP, nombre_archivo), VP)
            guardar_personas(os.path.join(carpeta_FP, nombre_archivo), FP)
            guardar_personas(os.path.join(carpeta_FN, nombre_archivo), FN)



In [21]:
def calcularResultados(umbral=0.7 ,ponderacion =  {1:1.5, 2:0.6, 3: 0.6,4:1, 5:0.8, 6:1, 7:0.8, 8:2.0}):
    """
    print(f"error numero -> {errorTotal}" )
    print(f"error distancia -> {distanciaTotal}" )
    print(f"resultados -> verdaderoPosiitvo {sumaVerdaderoPisitvo} falsoPositivo -> {sumaFalsoPositivo} falsonegativo -> {sumaFalsoNegativo} sobre -> {sumaTotal}")
    """
    crear_estadisticas(direccionYoloPredicciones , direccionCalcularPartes , umbral ,ponderacion )
    ficheroPrediccion =  os.listdir( direccionNuestroPredicciones )
    diccionarioResultados:"dict[str,int]" = {}
    diccionarioResultadosDistancia:"dict[str,float]" = {}

    listaCurposPrediccion = []
    listaCurposValidacion = []
    parte = []
    sumaTotal = 0
    sumaVerdaderoPisitvo = 0
    sumaFalsoPositivo = 0
    sumaFalsoNegativo = 0

    sumaErrores = 0
    sumaDistancias = 0.0
    distaciaPorArchivo = 0.0
    
    for archivo in ficheroPrediccion:
        listaCurposPrediccion = []
        listaCurposValidacion = []

        with open(f"{direccionNuestroPredicciones}/{archivo}", "r", encoding="utf-8") as f:
            for linea in f:
                parte = linea.replace("\n","").split(" ")
                if(parte[0] == "0"):
                    listaCurposPrediccion.append( parte )
                    sumaTotal += 1
                    sumaVerdaderoPisitvo += 1

        with open(f"{direccionFalsoPositivo}/{archivo}", "r", encoding="utf-8") as f:
            for linea in f:
                parte = linea.replace("\n","").split(" ")
                if(parte[0] == "0"):
                    listaCurposPrediccion.append( parte )
                    sumaTotal += 1
                    sumaFalsoPositivo += 1
        
        with open(f"{direccionFalsoNegativo}/{archivo}", "r", encoding="utf-8") as f:
            for linea in f:
                parte = linea.replace("\n","").split(" ")
                if(parte[0] == "0"):
                    listaCurposPrediccion.append( parte )
                    sumaTotal += 1
                    sumaFalsoNegativo += 1

        with open(f"{direccionNuestroValidacion}/{archivo}", "r", encoding="utf-8") as f:
            for linea in f:
                parte = linea.replace("\n","").split(" ")
                if(parte[0] == "0"):
                    listaCurposValidacion.append( parte )
        


        distaciaPorArchivo = 0.0
        for pred in listaCurposPrediccion:
            for valid in listaCurposValidacion:
                if estaDentro(valid, pred):
                    distacia = errorLocalizacion(valid , pred)
                    sumaDistancias += distacia
                    distaciaPorArchivo += distacia

        sumaErrores += abs( len(listaCurposPrediccion) - len(listaCurposValidacion) )
        diccionarioResultados[f"{archivo}"] =len(listaCurposPrediccion) - len(listaCurposValidacion)
        diccionarioResultadosDistancia[f"{archivo}"] = distaciaPorArchivo

    errorTotal = sumaErrores / len( ficheroPrediccion )
    distanciaTotal = sumaDistancias / len( ficheroPrediccion )
    diccionarioResultados["errorTotal"] = errorTotal
    diccionarioResultadosDistancia["errorTotal"] = distanciaTotal

    print(f"error numero -> {errorTotal}" )
    print(f"error distancia -> {distanciaTotal}" )

    print(f"resultados -> verdaderoPosiitvo {sumaVerdaderoPisitvo} falsoPositivo -> {sumaFalsoPositivo} falsonegativo -> {sumaFalsoNegativo} sobre -> {sumaTotal}")

In [22]:
def calcularResultadoSoloYolo():
    """
    Docstring for calcularResultado
    Misma función con las predicciones YOLO
    """
    listaCurposPrediccion = []
    listaCurposValidacion = []

    mach = True

    sumaErrores = 0
    sumaDistancias = 0.0
    distaciaPorArchivo = 0.0

    for archivo in ficheroYoloPredicciones:
        listaCurposPrediccion = []
        listaCurposValidacion = []
        with open(f"{direccionYoloPredicciones}/{archivo}", "r", encoding="utf-8") as f:
            for linea in f:
                parte = linea.replace("\n","").split(" ")
                if(parte[0] == "0"):
                    listaCurposPrediccion.append( parte )
        
        with open(f"{direccionNuestroValidacion}/{archivo}", "r", encoding="utf-8") as f:
            for linea in f:
                parte = linea.replace("\n","").split(" ")
                if(parte[0] == "0"):
                    listaCurposValidacion.append( parte )
        
        distaciaPorArchivo = 0.0
        sumaErrores = 0
        sumaDistancias = 0.0
        for pred in listaCurposPrediccion:
            for valid in listaCurposValidacion:
                if estaDentro(valid, pred):
                    distacia = errorLocalizacion(valid , pred)
                    sumaDistancias += distacia
                    distaciaPorArchivo += distacia
                    
            

        sumaErrores += abs( len(listaCurposPrediccion) - len(listaCurposValidacion) )
    errorTotal = sumaErrores / len( ficheroYoloPredicciones )
    distanciaTotal = sumaDistancias / len( ficheroYoloPredicciones )
    
    print(f"error numero -> {errorTotal}" )
    print(f"error distancia -> {distanciaTotal}" )

In [23]:
def crearDatoPercptron(ficheroEntrada ,direccionEntrada , direccionSalida):
    """
    Se guardan los datos a usar para entrenar el perceptron
    """
    mach:bool = False

    with open(f"ficheroEntrenar.txt","w") as f:
        f.writelines("VerdederoPositivo?:Torso,Hand,Foot,Upper Leg,Lower Leg,Upper Arm,Lower Arm,Head\n")

    for archivo in ficheroEntrada:
        listaCurposValidacion = []
        with open(f"{direccionSalida}/{archivo}", "r", encoding="utf-8") as f:
            for linea in f:
                parte = linea.replace("\n","").split(" ")
                if(parte[0] == "0"):
                    listaCurposValidacion.append( parte )

        with open(f"ficheroEntrenar.txt","a") as f:

            for personaPre , cuerpo in evaluar_imagenes_error(f"{direccionEntrada}/{archivo}").items():
                mach:bool = False
                for peronsaVal in listaCurposValidacion:
                    if estaDentro(peronsaVal, personaPre):
                        mach = True
                        break
                
                if mach:
                    f.writelines("1:")
                else:
                    f.writelines("0:")
                for i in range(1,9):
                    f.writelines(f"{cuerpo[i]},")
                f.writelines("\n")
    


In [24]:
import random
from sklearn.linear_model import Perceptron
import numpy as np
def entrenarPerceptron(numeroElemento = 300):
    """
    Entrenamos el perceptron
    """
    x_entrenar = []
    y_entrenar = []

    x0 = []
    x1 = []
    with open(f"ficheroEntrenar.txt","r") as f:
        next(f)
        for linea in f:
            aux = linea.replace("\n","")
            aux = aux.split(":")
            v = int(aux[0])
            aux = aux[1].split(",")[:-1]
            aux = [int(c) for c in aux]
            if v == 0:
                x0.append( aux )
            else:
                x1.append( aux)
            #x_entrenar.append( aux )


    for i in range( 0 ,numeroElemento ):
        y_entrenar.append( 0)
        x_entrenar.append( random.choice(x0) )

        y_entrenar.append( 1)
        x_entrenar.append( random.choice(x1) )

    print(y_entrenar)
    print(x_entrenar)

    print(len(y_entrenar))

    # Datos de ejemplo (8 entradas)
    X = np.array(x_entrenar)

    # Etiquetas (0 o 1)
    y = np.array(y_entrenar)

    # Crear el perceptrón
    clf = Perceptron(max_iter=10000, eta0=0.001, random_state=41)

    # Entrenar
    clf.fit(X, y)

    # Pesos y bias
    print("Pesos:", clf.coef_)
    print("Bias:", clf.intercept_)
    res = "{"
    idAux = 1
    for i in clf.coef_[0]:
        res+= f"{idAux}:{i},\n"
        idAux += 1

    print(res[:-2]+"}")

    # Predicciones
    y_pred = clf.predict(X)
    print("Predicciones:", y_pred)
    print("Predicciones:", len(y_pred))

    # Número de errores
    errores = np.sum(y != y_pred)
    print("Errores de clasificación:", errores)

In [25]:
crearDatoPercptron(ficheroYoloPrediccionesTrain , direccionYoloPrediccionestrain  , direccionTrain)
entrenarPerceptron(300)

[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 

In [26]:
ponderacion = { 1:0.008,
                2:0.002,
                3:0.009000000000000001,
                4:0.0,
                5:0.01800000000000001,
                6:0.006,
                7:-0.002,
                8:-0.008}
umbral=   -0.003
calcularResultados(umbral = umbral , ponderacion= ponderacion)


error numero -> 0.14
error distancia -> 0.08538757567720764
resultados -> verdaderoPosiitvo 322 falsoPositivo -> 19 falsonegativo -> 7 sobre -> 348


In [27]:
calcularResultadoSoloYolo()

error numero -> 0.0
error distancia -> 0.001557038445960327
